<a href="https://colab.research.google.com/github/junsu122/AI_Basic/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%94%A5%EB%9F%AC%EB%8B%9D_%EA%B8%B0%EC%B4%88/%EB%B6%84%EB%A5%98%20%EB%B0%8F%20%EB%B6%84%ED%8F%AC/%EC%9D%B4%EC%A7%84_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98_NLL_KL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 1. 데이터 로드 및 전처리
iris = load_iris()
X, y = iris.data, iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_tensor = torch.FloatTensor(X_scaled)

# 2. 모델 정의 (출력 노드 수를 인자로 받음)
class IrisNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )
    def forward(self, x):
        return self.fc(x)

# 3. 성능 비교 함수 (이진/다중 통합)
def compare_losses(num_classes, mode_name):
    # 데이터 준비 (이진 분류면 0, 1번 클래스만 추출)
    if num_classes == 2:
        mask = y < 2
        curr_X, curr_y = X_tensor[mask], torch.LongTensor(y[mask])
    else:
        curr_X, curr_y = X_tensor, torch.LongTensor(y)

    # KLDiv용 정답 분포 (One-hot)
    y_dist = F.one_hot(curr_y, num_classes=num_classes).float()

    # 두 모델 생성
    model_nll = IrisNet(num_classes)
    model_kl = IrisNet(num_classes)

    opt_nll = optim.Adam(model_nll.parameters(), lr=0.01)
    opt_kl = optim.Adam(model_kl.parameters(), lr=0.01)

    # 학습
    for epoch in range(100):
        # NLLLoss 학습
        opt_nll.zero_grad()
        out_nll = F.log_softmax(model_nll(curr_X), dim=1)
        loss_nll = nn.NLLLoss()(out_nll, curr_y)
        loss_nll.backward()
        opt_nll.step()

        # KLDiv 학습
        opt_kl.zero_grad()
        out_kl = F.log_softmax(model_kl(curr_X), dim=1)
        loss_kl = nn.KLDivLoss(reduction='batchmean')(out_kl, y_dist)
        loss_kl.backward()
        opt_kl.step()

    # 결과 분석 출력
    print(f"\n🚀 [{mode_name}] 성능 비교 보고서")
    print(f"{'Metric':<15} | {'NLLLoss Model':<15} | {'KLDiv Model':<15}")
    print("-" * 55)

    # 샘플 결과 (첫 번째 데이터 확인)
    with torch.no_grad():
        prob_nll = F.softmax(model_nll(curr_X[0:1]), dim=1)
        prob_kl = F.softmax(model_kl(curr_X[0:1]), dim=1)

    print(f"{'Final Loss':<15} | {loss_nll.item():<15.4f} | {loss_kl.item():<15.4f}")
    print(f"{'정답 확신도(%)':<15} | {prob_nll[0][curr_y[0]].item()*100:<15.2f} | {prob_kl[0][curr_y[0]].item()*100:<15.2f}")

# 4. 실행
compare_losses(num_classes=2, mode_name="이진 분류 (Setosa vs Versicolor)")
compare_losses(num_classes=3, mode_name="다중 분류 (All 3 Classes)")


🚀 [이진 분류 (Setosa vs Versicolor)] 성능 비교 보고서
Metric          | NLLLoss Model   | KLDiv Model    
-------------------------------------------------------
Final Loss      | 0.0022          | 0.0021         
정답 확신도(%)       | 99.89           | 99.97          

🚀 [다중 분류 (All 3 Classes)] 성능 비교 보고서
Metric          | NLLLoss Model   | KLDiv Model    
-------------------------------------------------------
Final Loss      | 0.0653          | 0.0842         
정답 확신도(%)       | 99.79           | 99.91          


In [9]:
import torch
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 1. 데이터 준비 (붓꽃 데이터)
iris = load_iris()
X = StandardScaler().fit_transform(iris.data)
X_tensor = torch.FloatTensor(X)

# 2. 이진 분류 예측 시뮬레이션 (Setosa vs Versicolor)
# 출력 노드가 2개인 경우
logits_binary = torch.tensor([[2.5, -1.2]]) # 모델이 내뱉은 원시 점수(Logits)
probs_binary = F.softmax(logits_binary, dim=1) # 확률로 변환

print("--- [1] 이진 분류 예측 (Binary Classification) ---")
print(f"원시 점수(Logits): {logits_binary.numpy()}")
print(f"변환된 확률: {probs_binary.numpy()}")
print(f"해석: 0번 클래스일 확률 {probs_binary[0][0]*100:.2f}%, 1번 클래스일 확률 {probs_binary[0][1]*100:.2f}%")
print(f"최종 결정: {torch.argmax(probs_binary).item()}번 품종")

print("\n" + "="*50 + "\n")

# 3. 다중 분류 예측 시뮬레이션 (Setosa, Versicolor, Virginica)
# 출력 노드가 3개인 경우
logits_multi = torch.tensor([[1.1, 3.8, -0.5]]) # 모델이 내뱉은 원시 점수(Logits)
probs_multi = F.softmax(logits_multi, dim=1) # 확률로 변환

print("--- [2] 다중 분류 예측 (Multi-class Classification) ---")
print(f"원시 점수(Logits): {logits_multi.numpy()}")
print(f"변환된 확률: {probs_multi.numpy()}")
for i, p in enumerate(probs_multi[0]):
    print(f"{iris.target_names[i]}: {p*100:.2f}%")
print(f"최종 결정: {iris.target_names[torch.argmax(probs_multi).item()]}")

--- [1] 이진 분류 예측 (Binary Classification) ---
원시 점수(Logits): [[ 2.5 -1.2]]
변환된 확률: [[0.975873   0.02412702]]
해석: 0번 클래스일 확률 97.59%, 1번 클래스일 확률 2.41%
최종 결정: 0번 품종


--- [2] 다중 분류 예측 (Multi-class Classification) ---
원시 점수(Logits): [[ 1.1  3.8 -0.5]]
변환된 확률: [[0.06218277 0.92526275 0.01255448]]
setosa: 6.22%
versicolor: 92.53%
virginica: 1.26%
최종 결정: versicolor
